# Задание по теме «Коллаборативная фильтрация»

ПАКЕТ SURPRISE

- используйте данные MovieLens 1M
- можно использовать любые модели из пакета
- получите RMSE на тестовом сете 0.87 и ниже

Комментарий преподавателя :

В ДЗ на датасет 1М может не хватить RAM. Можно сделать на 100K. Качество RMSE предлагаю считать на основе CrossValidation (5 фолдов), а не отложенном датасете.

In [1]:
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split, KFold, cross_validate

import pandas as pd

In [2]:
movies = pd.read_csv('../1_Введение и классификация рекомендательных систем/movies.csv')
ratings = pd.read_csv('../1_Введение и классификация рекомендательных систем/ratings.csv')

In [3]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [4]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})
dataset.head()

,uid,iid,rating
0,1.0,Toy Story (1995),4.0
1,5.0,Toy Story (1995),4.0
2,7.0,Toy Story (1995),4.5
3,15.0,Toy Story (1995),2.5
4,17.0,Toy Story (1995),4.5


In [5]:
len(dataset)

100836

In [6]:
ratings.rating.min(), ratings.rating.max()

(0.5, 5.0)

In [7]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

The available prediction algorithms are:

- knns.KNNBasic:	A basic collaborative filtering algorithm.
- knns.KNNWithMeans:	A basic collaborative filtering algorithm, taking into account the mean ratings of each user.
- knns.KNNWithZScore:	A basic collaborative filtering algorithm, taking into account the z-score normalization of each user.
- knns.KNNBaseline:	A basic collaborative filtering algorithm taking into account a baseline rating.


In [8]:
from surprise import KNNWithMeans, KNNBasic, KNNBaseline, KNNWithZScore

In [9]:
algo = KNNBaseline(k=40, sim_options={'name': 'pearson_baseline', 'user_based': True})

In [10]:
cross_val = cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=False, n_jobs=-1)
print(f"Cross validation scores: {cross_val['test_rmse']} \n Mean score: {cross_val['test_rmse'].mean():.3}")

Cross validation scores: [0.87915733 0.87194796 0.87974671 0.88240933 0.87115819] 
 Mean score: 0.877
